In [71]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

/home/iakovl0000/anaconda3/envs/stochastic/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [72]:
from bs4 import BeautifulSoup
import requests
import time

In [73]:
panther_names = {}

In [74]:
def get_panther_name(pantherid):
    if pantherid in panther_names.keys():
        return panther_names[pantherid];
    else: 
        r = requests.get("http://pantherdb.org/panther/family.do?clsAccession={}".format(pantherid))
        soup = BeautifulSoup(r.text, 'html.parser')
        family_name = soup.find(id="mainBody").table.tr.find("td",class_="mainText").text.strip()
        panther_names[pantherid] = family_name
        time.sleep(1)
        return family_name

In [75]:
panther_names

{}

In [76]:
datasets = [
    ("Gaublomme_GSE75109_TPM_clean", "clean_panther4march","geomean"),
    ("Gaublomme_GSE75110_TPM_clean","clean_panther4march","geomean"),
    ("Gaublomme_GSE75111_TPM_clean","clean_panther4march","geomean"),
#     "somatosensory_rpkm_suppl",
    ("kakadarov_tpm","clean_panther4march","geomean"),
    ("somatosensory_converted_into_tpm","clean_panther4march","geomean"),
    ("Dopaminergic_TPM_clean","clean_panther4march","geomean"),
    ("Rbp4_positive_cells","clean_panther4march","geomean"),
    ("Cheng_ES_TPM","clean_panther4march","geomean"),
    ("Alveolar_cells_Type_II_Merged_Batches","clean_panther4march","3max"),
    ("Alveolar_cells_Type_I_Merged_Batches","clean_panther4march","3max"),
#     ("Alveolar_cells_both_types","clean_panther4march","3max"),
    ("klein","clean_panther4march","3max"),
    ("hepat_TPM_yang_clean","clean_panther4march","geomean"),
    ("Yu_First_wave_endocrine_cells","clean_panther4march","geomean"),
#     ("lcl_european","HGNC_families"),
#     ("lcl_african","HGNC_families"),
#     ("cd4_zheng","HGNC_families"),
]



family_to_gene = pd.concat([
    pd.read_csv("clean_panther4march.csv", index_col="family_id"),
    pd.read_csv("HGNC_families.csv", index_col="family_id")
], sort=False)

In [77]:
# family_to_gene.loc[1492.0]

In [78]:
# pd.options.display.max_colwidth = 1000
pd.options.display.max_seq_items = 25

# Family-wise

* T-cell receptor beta-chain is present in all three Gaublomme sets (exclusive)
* Multiple histone families are conserved in all three Gaublomme sets (co-occurrent)
* Pcdh-clustered family is not shown as conserved (it is not exclusive in somatosensory and dopaminergic)
* A lot of Zinc finger proteins are conserved between all datasets (co-occurrent)

In [79]:
family_dfs = {x: pd.read_csv("{}/{}/results/family_IC.csv".format(x,y), index_col="family_id").dropna() for x, y, _ in datasets}

In [80]:
def get_lower(df):
    return df[df.ic <= df.ic.quantile(0.025)].index.values

def get_upper(df):
    return df[df.ic >= df.ic.quantile(0.975)].index.values

def get_intersection(l1, l2):
    return set(l1).intersection(l2)

In [81]:
dataset_names = [x[0] for x in datasets]

In [82]:
# Overlap display formaters
def format_family_names(x):
    if x is None or len(x) == 0:
        return ""
    _x = list(x.copy())
    s = "\n".join(_x[:10])
    if(len(_x) > 10):
        s += "\n+{} families".format(len(_x)-10)
    return s

def format_gene_names(x):
    if x is None or len(x) == 0:
        return ""
    
    s = ""
    for genes in x[:5]:
        genes_s = ",".join(genes[:5])
        if (len(genes) > 5):
            genes_s += " +{} genes".format(len(genes)-5)
        s += genes_s + "\n"
    if(len(x) > 5):
        s += "+{} families".format(len(x)-5)
    return s.strip()
    

In [83]:
family_overlap_lower = pd.DataFrame([[None if x == y else get_intersection(get_lower(family_dfs[x]),get_lower(family_dfs[y])) for y, _, _ in datasets] for x, _, _ in datasets], index=dataset_names, columns=dataset_names)

In [84]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_seq_items = 5
writer = pd.ExcelWriter("overlap_family_lower.xlsx")
# display(family_overlap_lower.applymap(lambda x: "" if x is None else ",".join(list(x))))
family_overlap_lower.applymap(lambda x: "" if x is None else ",".join(list(x))).to_csv("overlap_family_lower_raw.csv")
display(family_overlap_lower.applymap(format_family_names))
display(family_overlap_lower.progress_applymap(lambda x: [get_panther_name(_x) for _x in x] if x is not None else None).applymap(format_family_names))
display(family_overlap_lower.progress_applymap(lambda x: [family_to_gene.loc[np.int(family) if isinstance(family, np.float64) else family].gene_symbol for family in x] if x is not None else []).applymap(format_gene_names))
display(family_overlap_lower.progress_applymap(lambda x: len(x) if x is not None else np.nan))

family_overlap_lower.applymap(format_family_names).to_excel(writer,"by IDs")
family_overlap_lower.applymap(lambda x: [get_panther_name(_x) for _x in x] if x is not None else None).applymap(format_family_names).to_excel(writer,"by family names")
family_overlap_lower.applymap(lambda x: [family_to_gene.loc[np.int(family) if isinstance(family, np.float64) else family].gene_symbol for family in x] if x is not None else []).applymap(format_gene_names).to_excel(writer,"by genes")
family_overlap_lower.applymap(lambda x: len(x) if x is not None else np.nan).to_excel(writer,"count by families")
writer.save()

,Gaublomme_GSE75109_TPM_clean,Gaublomme_GSE75110_TPM_clean,Gaublomme_GSE75111_TPM_clean,kakadarov_tpm,somatosensory_converted_into_tpm,Dopaminergic_TPM_clean,Rbp4_positive_cells,Cheng_ES_TPM,Alveolar_cells_Type_II_Merged_Batches,Alveolar_cells_Type_I_Merged_Batches,klein,hepat_TPM_yang_clean,Yu_First_wave_endocrine_cells
Gaublomme_GSE75109_TPM_clean,,PTHR23268\nPTHR11636,PTHR24089\nPTHR23268\nPTHR19375,PTHR12587\nPTHR11453,PTHR11453\nPTHR19375,,,,,,PTHR13771,,
Gaublomme_GSE75110_TPM_clean,PTHR23268\nPTHR11636,,PTHR12570\nPTHR23268,PTHR45615\nPTHR11969,PTHR12570\nPTHR42884,,PTHR11969,PTHR45615,,PTHR12268,,PTHR12570,PTHR10901
Gaublomme_GSE75111_TPM_clean,PTHR24089\nPTHR23268\nPTHR19375,PTHR12570\nPTHR23268,,,PTHR18952\nPTHR12570\nPTHR25465\nPTHR19375,,,PTHR18952,,PTHR13715,,PTHR18952\nPTHR12570\nPTHR23122,PTHR23122
kakadarov_tpm,PTHR12587\nPTHR11453,PTHR45615\nPTHR11969,,,PTHR11453,,PTHR11969,PTHR45615,PTHR10361,,,PTHR10361,PTHR45627
somatosensory_converted_into_tpm,PTHR11453\nPTHR19375,PTHR12570\nPTHR42884,PTHR18952\nPTHR12570\nPTHR25465\nPTHR19375,PTHR11453,,,PTHR10075,PTHR18952,,,,PTHR18952\nPTHR12570,PTHR10269
Dopaminergic_TPM_clean,,,,,,,PTHR12844,PTHR14002,PTHR24543,PTHR24061,,,
Rbp4_positive_cells,,PTHR11969,,PTHR11969,PTHR10075,PTHR12844,,PTHR11890,PTHR13388,,,,PTHR19384
Cheng_ES_TPM,,PTHR45615,PTHR18952,PTHR45615,PTHR18952,PTHR14002,PTHR11890,,PTHR14132\nPTHR14949,,,PTHR18952\nPTHR14949\nPTHR16024,PTHR14132\nPTHR19443
Alveolar_cells_Type_II_Merged_Batches,,,,PTHR10361,,PTHR24543,PTHR13388,PTHR14132\nPTHR14949,,PTHR13869\nPTHR13817,,PTHR14949\nPTHR10361\nPTHR24241,PTHR11733\nPTHR13817\nPTHR14132
Alveolar_cells_Type_I_Merged_Batches,,PTHR12268,PTHR13715,,,PTHR24061,,,PTHR13869\nPTHR13817,,,,PTHR13817


KeyboardInterrupt: 

In [ ]:
family_overlap_upper = pd.DataFrame([[None if x == y else get_intersection(get_upper(family_dfs[x]),get_upper(family_dfs[y])) for y, _, _ in datasets] for x, _, _ in datasets], index=dataset_names, columns=dataset_names)

In [ ]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_seq_items = 5
writer = pd.ExcelWriter("overlap_family_upper.xlsx")
display(family_overlap_upper.applymap(format_family_names))
display(family_overlap_upper.progress_applymap(lambda x: [get_panther_name(_x) for _x in x] if x is not None else None).applymap(format_family_names))
display(family_overlap_upper.progress_applymap(lambda x: [family_to_gene.loc[np.int(family) if isinstance(family, np.float64) else family].gene_symbol for family in x] if x is not None else []).applymap(format_gene_names))
display(family_overlap_upper.progress_applymap(lambda x: len(x) if x is not None else np.nan))

family_overlap_upper.applymap(format_family_names).to_excel(writer,"by IDs")
family_overlap_upper.applymap(lambda x: [get_panther_name(_x) for _x in x] if x is not None else None).applymap(format_family_names).to_excel(writer,"by family names")
family_overlap_upper.applymap(lambda x: [family_to_gene.loc[np.int(family) if isinstance(family, np.float64) else family].gene_symbol for family in x] if x is not None else []).applymap(format_gene_names).to_excel(writer,"by genes")
family_overlap_upper.applymap(lambda x: len(x) if x is not None else np.nan).to_excel(writer,"count by families")
writer.save()

In [ ]:
# print("Pcdh clustered, IC vs. 2.5 percentile")
# display("somatosensory_converted_into_tpm")
# display(family_dfs["somatosensory_converted_into_tpm"].loc["PTHR24028_clustered"].ic)
# display(family_dfs["somatosensory_converted_into_tpm"].ic.quantile(0.025))

# # display("somatosensory_rpkm_suppl")
# # display(family_dfs["somatosensory_rpkm_suppl"].loc["PTHR24028_clustered"].ic)
# # display(family_dfs["somatosensory_rpkm_suppl"].ic.quantile(0.025))

# display("dopaminergic")
# display(family_dfs["dopaminergic"].loc["PTHR24028_clustered"].ic)
# display(family_dfs["dopaminergic"].ic.quantile(0.025))

# display("Rbp4_positive_cells")
# display(family_dfs["Rbp4_positive_cells"].loc["PTHR24028_clustered"].ic)
# display(family_dfs["Rbp4_positive_cells"].ic.quantile(0.025))

# Chromosome-wise

* Some Pcdh stretches are overlapping between different sets of neurons (exclusive)
* The highest overlap of the co-occurrent in Gaublomme sets (majorly comprised of histone genes) 

In [85]:
from tqdm import tqdm_notebook 

In [86]:
chr_names = list(range(1,20)) + ["X", "Y"]
chrs = {x: pd.read_csv("somatosensory_converted_into_tpm/intermediate/chr{}_filtered.csv".format(x)) for x in chr_names}

In [87]:
def load_chromosome_wise(dataset, dich_type):
#     dich_type = "geomean"
    chr_names = list(range(1,20)) + ["X", "Y"]
    ic_dfs = [pd.read_csv("{}/results/{}/stage1_chr{}_IC.csv".format(dataset,dich_type,x)) for x in chr_names]
    chrs = {x: pd.read_csv("{}/intermediate/chr{}_filtered.csv".format(dataset, x)) for x in chr_names}
    filtered_genes = pd.read_csv("{}/intermediate/{}/filtered_dichotomised_genes.csv".format(dataset, dich_type))
    for i, name in zip(range(len(ic_dfs)), chr_names):
        ic_dfs[i].loc[:,"chromosome"] = name

    ic_df = pd.concat(ic_dfs, sort=False).reset_index(drop=True)
    return ic_df[ic_df.n_genes >= ic_df.stretch * 6 / 7]

In [88]:
def get_genes(start, end, chromosome):
    return chrs[chromosome].loc[start:end, "Name"].values

def get_all_genes(l):
    if l is None:
        return set()
    return sorted(set(np.concatenate([get_genes(*x) for x in l])))

In [89]:
chromosome_dfs = {x: load_chromosome_wise(x, dich_type) for x, _, dich_type in tqdm_notebook(datasets)}

/home/iakovl0000/anaconda3/envs/stochastic/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [90]:
chromosome_dfs["Gaublomme_GSE75109_TPM_clean"]

,start,end,stretch,n_genes,obs_var,pb_var,ic,mean_expression,chromosome
29,29.0,36.0,7.0,6.0,0.488687,0.420475,1.162226,1.402878,1
30,30.0,37.0,7.0,6.0,0.231988,0.239429,0.968925,0.640288,1
127,127.0,134.0,7.0,6.0,0.996664,0.875006,1.139036,2.057554,1
128,128.0,135.0,7.0,6.0,1.211135,0.980281,1.235499,2.187050,1
129,129.0,136.0,7.0,6.0,1.134397,0.903576,1.255452,1.625899,1
...,...,...,...,...,...,...,...,...,...
69616,701.0,715.0,14.0,12.0,1.050360,0.928627,1.131089,2.791367,X
69850,935.0,949.0,14.0,12.0,1.937963,1.499301,1.292577,2.597122,X
69851,936.0,950.0,14.0,13.0,2.191012,1.660163,1.319758,2.798561,X
69852,937.0,951.0,14.0,13.0,2.191012,1.660163,1.319758,2.798561,X


In [91]:
def get_lower(df):
#     print(df[df.ic <= df.ic.quantile(0.025)].shape[0])
    slices = []
    for stretch in [7, 14, 21]:
        _slice = df[df.stretch == stretch]
#         print(_slice[_slice.ic <= _slice.ic.quantile(0.025)].shape[0])
        slices += [_slice[_slice.ic <= _slice.ic.quantile(0.025)]]
    return pd.concat(slices).loc[:, ["start", "end", "chromosome"]].values

def get_upper(df):
    slices = []
    for stretch in [7, 14, 21]:
        _slice = df[df.stretch == stretch]
        slices += [_slice[_slice.ic >= _slice.ic.quantile(0.975)]]
    return pd.concat(slices).loc[:, ["start", "end", "chromosome"]].values

def get_intersection(l1, l2):
    intersection = []
    for x in l1:
        for y in l2:
            if np.all(x == y):
                intersection += [x]
    if len(intersection) == 0:
        return None
    return np.stack(intersection)


In [92]:
dataset_names = [x for x, _, _ in datasets]

In [93]:
chromosome_overlap_lower = pd.DataFrame([[None if x == y else get_intersection(get_lower(chromosome_dfs[x]),get_lower(chromosome_dfs[y])) for y in dataset_names] for x in dataset_names], index=dataset_names, columns=dataset_names)

In [94]:
def format_stretches(x):
    if x is None or len(x) == 0:
        return ""
    _x = list(x.copy())
    s = "\n".join([str(__x) for __x in _x[:5]])
    if(len(_x) > 5):
        s += "\n+{} stretches".format(len(_x)-5)
    return s

def format_stretch_genes(x):
    if x is None or len(x) == 0:
        return ""
    _x = list(x.copy())
    s = "\n".join([str(__x) for __x in _x[:25]])
    if(len(_x) > 25):
        s += "\n+{} genes".format(len(_x)-25)
    return s


In [95]:
import pickle
with open("overlap_chromosome_lower_raw.pickle", "wb") as f:
    pickle.dump(chromosome_overlap_lower, f)

writer = pd.ExcelWriter("overlap_chromosome_lower.xlsx")
chromosome_overlap_lower.applymap(get_all_genes).applymap(format_stretch_genes).to_excel(writer,"by genes")
chromosome_overlap_lower.applymap(lambda x: len(x) if x is not None else np.nan).to_excel(writer,"by count (stretches)")
chromosome_overlap_lower.applymap(get_all_genes).applymap(lambda x: len(x) if x is not None else np.nan).to_excel(writer,"by count (genes)")
chromosome_overlap_lower.applymap(format_stretches).to_excel(writer,"by locations")
writer.save()

In [96]:
chromosome_overlap_lower.applymap(format_stretches)

,Gaublomme_GSE75109_TPM_clean,Gaublomme_GSE75110_TPM_clean,Gaublomme_GSE75111_TPM_clean,kakadarov_tpm,somatosensory_converted_into_tpm,Dopaminergic_TPM_clean,Rbp4_positive_cells,Cheng_ES_TPM,Alveolar_cells_Type_II_Merged_Batches,Alveolar_cells_Type_I_Merged_Batches,klein,hepat_TPM_yang_clean,Yu_First_wave_endocrine_cells
Gaublomme_GSE75109_TPM_clean,,[814.0 821.0 3],,[68.0 75.0 10],,,,,,[68.0 75.0 10]\n[366.0 380.0 13],,,[53.0 60.0 'X']
Gaublomme_GSE75110_TPM_clean,[814.0 821.0 3],,[769.0 776.0 5]\n[97.0 104.0 8]\n[242.0 256.0 6]\n[244.0 258.0 6],[791.0 798.0 3],,[639.0 646.0 11],,,,,[96.0 103.0 19],,[465.0 472.0 13]
Gaublomme_GSE75111_TPM_clean,,[769.0 776.0 5]\n[97.0 104.0 8]\n[242.0 256.0 6]\n[244.0 258.0 6],,,,[655.0 662.0 'X'],,,,[1990.0 1997.0 2]\n[1186.0 1193.0 5]\n[382.0 389.0 13]\n[603.0 610.0 15]\n[946.0 953.0 17],,,
kakadarov_tpm,[68.0 75.0 10],[791.0 798.0 3],,,[752.0 759.0 2],[531.0 538.0 18]\n[461.0 475.0 17],[326.0 333.0 7],,[427.0 434.0 1]\n[154.0 161.0 2]\n[1872.0 1879.0 2]\n[290.0 297.0 5]\n[623.0 630.0 9]\n+7 stretches,[34.0 41.0 6]\n[343.0 350.0 6]\n[68.0 75.0 10]\n[591.0 598.0 14]\n[897.0 904.0 17]\n+2 stretches,[478.0 485.0 2]\n[372.0 379.0 4]\n[362.0 369.0 7]\n[363.0 370.0 7]\n[393.0 400.0 7]\n+7 stretches,,[1239.0 1246.0 9]\n[927.0 934.0 10]\n[627.0 634.0 11]
somatosensory_converted_into_tpm,,,,[752.0 759.0 2],,[1458.0 1465.0 2]\n[932.0 939.0 7]\n[202.0 209.0 18]\n[203.0 210.0 18]\n[341.0 348.0 'X']\n+1 stretches,[398.0 405.0 3]\n[203.0 210.0 18]\n[204.0 211.0 18]\n[205.0 212.0 18]\n[206.0 213.0 18]\n+13 stretches,[591.0 598.0 2],[632.0 639.0 14]\n[1124.0 1131.0 17]\n[622.0 636.0 4],[1393.0 1400.0 11],[202.0 209.0 18]\n[203.0 210.0 18]\n[204.0 211.0 18]\n[199.0 213.0 18]\n[200.0 214.0 18]\n+10 stretches,,[1456.0 1463.0 2]\n[932.0 939.0 7]
Dopaminergic_TPM_clean,,[639.0 646.0 11],[655.0 662.0 'X'],[531.0 538.0 18]\n[461.0 475.0 17],[1458.0 1465.0 2]\n[932.0 939.0 7]\n[202.0 209.0 18]\n[203.0 210.0 18]\n[341.0 348.0 'X']\n+1 stretches,,[53.0 60.0 7]\n[203.0 210.0 18]\n[223.0 230.0 18]\n[224.0 231.0 18]\n[200.0 214.0 18]\n+3 stretches,,[202.0 209.0 2]\n[1060.0 1067.0 6]\n[531.0 538.0 18]\n[395.0 409.0 1]\n[1555.0 1569.0 11]\n+7 stretches,[1840.0 1847.0 7]\n[763.0 770.0 9]\n[213.0 234.0 18]\n[214.0 235.0 18]\n[215.0 236.0 18]\n+1 stretches,[403.0 410.0 1]\n[406.0 413.0 1]\n[201.0 208.0 18]\n[202.0 209.0 18]\n[203.0 210.0 18]\n+1 stretches,[700.0 707.0 10]\n[1287.0 1308.0 1],[400.0 407.0 1]\n[1038.0 1045.0 2]\n[932.0 939.0 7]\n[94.0 101.0 11]\n[704.0 711.0 'X']\n+5 stretches
Rbp4_positive_cells,,,,[326.0 333.0 7],[398.0 405.0 3]\n[203.0 210.0 18]\n[204.0 211.0 18]\n[205.0 212.0 18]\n[206.0 213.0 18]\n+13 stretches,[53.0 60.0 7]\n[203.0 210.0 18]\n[223.0 230.0 18]\n[224.0 231.0 18]\n[200.0 214.0 18]\n+3 stretches,,[535.0 542.0 1],[218.0 232.0 18]\n[222.0 236.0 18]\n[223.0 237.0 18],[535.0 542.0 1]\n[1444.0 1451.0 11]\n[560.0 567.0 19],[203.0 210.0 18]\n[204.0 211.0 18]\n[199.0 213.0 18]\n[200.0 214.0 18]\n[201.0 215.0 18]\n+4 stretches,,[979.0 986.0 5]
Cheng_ES_TPM,,,,,[591.0 598.0 2],,[535.0 542.0 1],,,[535.0 542.0 1],,,
Alveolar_cells_Type_II_Merged_Batches,,,,[427.0 434.0 1]\n[154.0 161.0 2]\n[1872.0 1879.0 2]\n[290.0 297.0 5]\n[623.0 630.0 9]\n+7 stretches,[632.0 639.0 14]\n[1124.0 1131.0 17]\n[622.0 636.0 4],[202.0 209.0 2]\n[1060.0 1067.0 6]\n[531.0 538.0 18]\n[395.0 409.0 1]\n[1555.0 1569.0 11]\n+7 stretches,[218.0 232.0 18]\n[222.0 236.0 18]\n[223.0 237.0 18],,,[113.0 120.0 1]\n[114.0 121.0 1]\n[429.0 436.0 1]\n[1298.0 1305.0 1]\n[1323.0 1330.0 6]\n+87 stretches,,[619.0 626.0 2]\n[620.0 627.0 2],[489.0 496.0 1]\n[675.0 682.0 7]\n[404.0 418.0 15]\n[214.0 228.0 18]\n[215.0 229.0 18]\n+2 stretches
Alveolar_cells_Type_I_Merged_Batches,[68.0 75.0 10]\n[366.0 380.0 13],,[1990.0 1997.0 2]\n[1186.0 1193.0 5]\n[382.0 389.0 13]\n[603.0 610.0 15]\n[946.0 953.0 17],[34.0 41.0 6]\n[343.0 350.0 6]\n[68.0 75.0 10]\n[591.0 598.0 14]\n[897.0 904.0 17]\n+2 stretches,[1393.0 1400.0 11],[1840.0 1847.0 7]\n[763.0 770.0 9]

In [97]:
for _x in range(chromosome_overlap_lower.shape[0]):
    for _y in range(chromosome_overlap_lower.shape[1]):
        x = (chromosome_overlap_lower.iloc[_x,_y])
        if x is None:
            continue
        x = pd.DataFrame(np.hstack([x, np.reshape(x[:,1] - x[:,0], (-1,1))]), columns=["a", "b", "chr", "size"])
        x = x.sort_values("size", ascending=False)
        for i in range(x.shape[0]):
            y = x.iloc[i]
            z = x.loc[(x.chr == y.chr) & (x.a <= y.a) & (x.b >= y.b) & (x.size < y.size)]
            if(z.shape[0] != 0):
                print(_x,_y,i)
                print(z)
                break

In [98]:

chromosome_overlap_lower.applymap(get_all_genes).applymap(format_stretch_genes)

,Gaublomme_GSE75109_TPM_clean,Gaublomme_GSE75110_TPM_clean,Gaublomme_GSE75111_TPM_clean,kakadarov_tpm,somatosensory_converted_into_tpm,Dopaminergic_TPM_clean,Rbp4_positive_cells,Cheng_ES_TPM,Alveolar_cells_Type_II_Merged_Batches,Alveolar_cells_Type_I_Merged_Batches,klein,hepat_TPM_yang_clean,Yu_First_wave_endocrine_cells
Gaublomme_GSE75109_TPM_clean,,Aknad1\nFam102b\nFndc7\nHenmt1\nPrpf38b\nSlc25a24\nSlc25a54\nStxbp3,,4930444F02Rik\nAbracl\nArfgef3\nCcdc28a\nEct2l\nHebp2\nNhsl1\nReps1,,,,,,1700029N11Rik\n4930444F02Rik\nA330076C08Rik\nAbracl\nArfgef3\nAtxn1\nCcdc28a\nCd83\nDtnbp1\nEct2l\nGfod1\nGmpr\nHebp2\nJarid2\nMcur1\nMylip\nNhsl1\nNol7\nRanbp9\nReps1\nRnf182\nSirt5\nTbc1d7,,,Eras\nGata1\nGlod5\nHdac6\nPcsk1n\nSuv39h1\nWas\nWdr13
Gaublomme_GSE75110_TPM_clean,Aknad1\nFam102b\nFndc7\nHenmt1\nPrpf38b\nSlc25a24\nSlc25a54\nStxbp3,,Acacb\nAlkbh2\nArhgef10\nCln8\nCoprs\nDlgap2\nErich1\nFbxo25\nFoxn4\nKbtbd11\nKctd10\nMyo1h\nTdrp\nTrbv12-1\nTrbv12-2\nTrbv13-1\nTrbv13-2\nTrbv13-3\nTrbv14\nTrbv15\nTrbv16\nTrbv17\nTrbv19\nTrbv20\nTrbv21\n+8 genes,Amigo1\nAmpd2\nAtxn7l2\nCyb561d1\nGnai3\nGnat2\nGpr61\nGstm4,,Atp1b2\nDnah2\nEfnb3\nKdm6b\nShbg\nTmem88\nTrp53\nWrap53,,,,,1700020D05Rik\nAp5b1\nCfl1\nEfemp2\nMus81\nOvol1\nRnaseh2c\nSnx32,,Eif4e1b\nFgfr4\nHk3\nSncb\nTspan17\nUimc1\nUnc5a\nZfp346
Gaublomme_GSE75111_TPM_clean,,Acacb\nAlkbh2\nArhgef10\nCln8\nCoprs\nDlgap2\nErich1\nFbxo25\nFoxn4\nKbtbd11\nKctd10\nMyo1h\nTdrp\nTrbv12-1\nTrbv12-2\nTrbv13-1\nTrbv13-2\nTrbv13-3\nTrbv14\nTrbv15\nTrbv16\nTrbv17\nTrbv19\nTrbv20\nTrbv21\n+8 genes,,,,2610002M06Rik\n2810403D21Rik\nBrwd3\nGpr174\nItm2a\nP2ry10b\nTbx22\nTent5d,,,,Amz1\nArfrp1\nBrat1\nCap2\nCard11\nCd70\nCrb3\nDennd1c\nDennd6b\nFam8a1\nGmeb2\nGna12\nGrifin\nHdac10\nHelz2\nIqce\nKdm1b\nKhsrp\nKif13a\nLfng\nLime1\nMapk11\nMapk12\nMir6958\nMir7118\n+15 genes,,,
kakadarov_tpm,4930444F02Rik\nAbracl\nArfgef3\nCcdc28a\nEct2l\nHebp2\nNhsl1\nReps1,Amigo1\nAmpd2\nAtxn7l2\nCyb561d1\nGnai3\nGnat2\nGpr61\nGstm4,,,4930443O20Rik\nAplnr\nLrrc55\nOlfr987\nOlfr988\nP2rx3\nSsrp1\nTnks1bp1,Atp5a1\nCyp4f13\nCyp4f14\nCyp4f15\nCyp4f16\nCyp4f17\nCyp4f37\nCyp4f39\nCyp4f40\nEpg5\nF830208F22Rik\nPglyrp2\nPstpip2\nSetbp1\nSiglec15\nSlc14a1\nSlc14a2\nZfp472\nZfp799\nZfp811\nZfp870\nZfp871\nZfp952,Bloc1s3\nExoc3l2\nGemin7\nMark4\nNkpd1\nPpp1r37\nTrappc6a\nZfp296,,1110038F14Rik\n1700025B11Rik\n1700072B07Rik\n1700109G14Rik\n1700109K24Rik\n4930426L09Rik\n4930447M23Rik\n4930487D11Rik\n4930513D17Rik\n4930515B02Rik\n4933415F23Rik\n6720483E21Rik\nAkap12\nAmer2\nApol10a\nApol10b\nApol11a\nApol11b\nApol6\nApol7a\nApol7b\nApol7c\nApol7e\nApol8\nApol9a\n+113 genes,1110038F14Rik\n1700007K13Rik\n1700025B11Rik\n1700109K24Rik\n4930444F02Rik\nA430035B10Rik\nAA545190\nAbracl\nAk8\nApol10a\nApol10b\nApol11a\nApol11b\nApol6\nApol7a\nApol7b\nApol7c\nApol7e\nApol8\nApol9a\nApol9b\nArfgef3\nBcl2l2\nCcdc28a\nCel\n+52 genes,1700028J19Rik\n2410002F23Rik\nKlk1\nKlk1b11\nKlk1b16\nKlk1b21\nKlk1b22\nKlk1b24\nKlk1b26\nKlk1b27\nKlk1b3\nKlk1b4\nKlk1b5\nKlk1b9\nMup17\nMup18\nMup19\nMup20\nMup21\nMup3\nMup5\nNlrp4e\nOlfr341\nOlfr342\nOlfr344\n+36 genes,,Alox12b\nAlox8\nAloxe3\nArhgap9\nCntrob\nEif1b\nEntpd3\nGli1\nGucy2e\nHes7\nInhbc\nInhbe\nKcnab3\nMobp\nMyrip\nNdufa4l2\nR3hdm2\nRpl14\nRpsa\nShmt2\nSlc25a38\nSnora62\nStac3\nTrappc1
somatosensory_converted_into_tpm,,,,4930443O20Rik\nAplnr\nLrrc55\nOlfr987\nOlfr988\nP2rx3\nSsrp1\nTnks1bp1,,4930550L24Rik\nAbcc6\nAbcc8\nAtp11c\nBmp2\nCcdc114\nCrls1\nEmp3\nF9\nFermt1\nFgf13\nGpr101\nHao1\nKcnj11\nLrrn4\nMcf2\nMcm8\nMir3090\nMir504\nNomo1\nPcdha1\nPcdha10\nPcdha11\nPcdha12\nPcdha2\n+14 genes,Bglap\nBglap2\nBglap3\nDnd1\nHars\nHars2\nPaqr6\nPcdha1\nPcdha10\nPcdha11\nPcdha12\nPcdha2\nPcdha3\nPcdha4\nPcdha5\nPcdha6\nPcdha7\nPcdha8\nPcdha9\nPcdhac1\nPcdhac2\nPcdhb1\nPcdhb2\nPcdhb3\nPcdhga1\n+26 genes,Dpp4\nFap\nFign\nGca\nGcg\nIfih1\nKcnh7\nSlc4a10,4930480K15Rik\n4933400B14Rik\nAdcy4\nCideb\nFoxd2\nFoxd2os\nFshr\nGtf2a1l\nLhcgr\nLtb4r1\nLtb4r2\nNfatc4\nNop9\nNrxn1\nNynrin\nPpp1r21\nRipk3\nSkint1\nSkint10\nSkint11

In [99]:
chromosome_overlap_lower.applymap(lambda x: len(x) if x is not None else np.nan)

,Gaublomme_GSE75109_TPM_clean,Gaublomme_GSE75110_TPM_clean,Gaublomme_GSE75111_TPM_clean,kakadarov_tpm,somatosensory_converted_into_tpm,Dopaminergic_TPM_clean,Rbp4_positive_cells,Cheng_ES_TPM,Alveolar_cells_Type_II_Merged_Batches,Alveolar_cells_Type_I_Merged_Batches,klein,hepat_TPM_yang_clean,Yu_First_wave_endocrine_cells
Gaublomme_GSE75109_TPM_clean,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0
Gaublomme_GSE75110_TPM_clean,1.0,NaN,4.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0
Gaublomme_GSE75111_TPM_clean,NaN,4.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN
kakadarov_tpm,1.0,1.0,NaN,NaN,1.0,2.0,1.0,NaN,12.0,7.0,12.0,NaN,3.0
somatosensory_converted_into_tpm,NaN,NaN,NaN,1.0,NaN,6.0,18.0,1.0,3.0,1.0,15.0,NaN,2.0
Dopaminergic_TPM_clean,NaN,1.0,1.0,2.0,6.0,NaN,8.0,NaN,12.0,6.0,6.0,2.0,10.0
Rbp4_positive_cells,NaN,NaN,NaN,1.0,18.0,8.0,NaN,1.0,3.0,3.0,9.0,NaN,1.0
Cheng_ES_TPM,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
Alveolar_cells_Type_II_Merged_Batches,NaN,NaN,NaN,12.0,3.0,12.0,3.0,NaN,NaN,92.0,NaN,2.0,7.0
Alveolar_cells_Type_I_Merged_Batches,2.0,NaN,5.0,7.0,1.0,6.0,3.0,1.0,92.0,NaN,NaN,NaN,3.0


In [100]:
chromosome_overlap_upper = pd.DataFrame([[None if x == y else get_intersection(get_upper(chromosome_dfs[x]),get_upper(chromosome_dfs[y])) for y in dataset_names] for x in dataset_names], index=dataset_names, columns=dataset_names)

In [101]:
writer = pd.ExcelWriter("overlap_chromosome_upper.xlsx")
chromosome_overlap_upper.applymap(get_all_genes).applymap(format_stretch_genes).to_excel(writer,"by genes")
chromosome_overlap_upper.applymap(lambda x: len(x) if x is not None else np.nan).to_excel(writer,"by count (stretches)")
chromosome_overlap_upper.applymap(get_all_genes).applymap(lambda x: len(x) if x is not None else np.nan).to_excel(writer,"by count (genes)")
chromosome_overlap_upper.applymap(format_stretches).to_excel(writer,"by locations")
writer.save()

In [102]:
# pd.options.display.max_colwidth = 1000
pd.options.display.max_seq_items = 100
chromosome_overlap_upper.applymap(get_all_genes)

,Gaublomme_GSE75109_TPM_clean,Gaublomme_GSE75110_TPM_clean,Gaublomme_GSE75111_TPM_clean,kakadarov_tpm,somatosensory_converted_into_tpm,Dopaminergic_TPM_clean,Rbp4_positive_cells,Cheng_ES_TPM,Alveolar_cells_Type_II_Merged_Batches,Alveolar_cells_Type_I_Merged_Batches,klein,hepat_TPM_yang_clean,Yu_First_wave_endocrine_cells
Gaublomme_GSE75109_TPM_clean,{},"[9930111J21Rik1, 9930111J21Rik2, H1f1, H1f2, H1f3, H1f4, H1f6, H2ac11, H2ac12, H2ac18, H2ac19, H2ac20, H2ac4, H2ac6, H2ac7, H2ac8, H2bc11, H2bc12, H2bc18, H2bc21, H2bc3, H2bc4, H2bc6, H2bc7, H2bc8, H3c1, H3c13, H3c14, H3c15, H3c2, H3c3, H3c4, H3c6, H3c7, H4c1, H4c14, H4c2, H4c3, H4c4, H4c6, H4c9, Hfe, Ifi47, Irgm1, Olfr1395, Olfr1396, Olfr56, Pom121l2, Prss16, Psme2b, Slc17a2, Tgtp1, Tgtp2, Trim38, Trim41, Trim7, Vmn1r188, Vmn1r189, Zfp184]","[9930111J21Rik1, 9930111J21Rik2, A530032D15Rik, Ampd1, Arrdc5, Bcas2, C130026I21Rik, Cchcr1, Csde1, Dennd2c, Fbxo36, Fem1a, Gm7609, H1f1, H1f2, H1f3, H1f4, H1f6, H2-Q10, H2-Q2, H2-Q4, H2-Q6, H2-Q7, H2ac11, H2ac12, H2ac18, H2ac19, H2ac20, H2ac4, H2ac6, H2ac7, H2ac8, H2bc11, H2bc12, H2bc18, H2bc21, H2bc3, H2bc4, H2bc6, H2bc7, H2bc8, H3c1, H3c13, H3c14, H3c15, H3c2, H3c3, H3c4, H3c6, H3c7, H4c1, H4c14, H4c2, H4c3, H4c4, H4c6, H4c9, Hfe, Ifi47, Irgm1, Kdm4b, Mir6977, Nras, Olfr1395, Olfr1396, Olfr56, Plin3, Pom121l2, Pou5f1, Prss16, Psme2b, Psors1c2, Ptprs, Sike1, Slc16a14, Slc17a2, Sp100, Sp110, Sp140, Syt6, Tcf19, Tgtp1, Tgtp2, Ticam1, Trim33, Trim38, Trim41, Trim7, Uhrf1, Vmn1r188, Vmn1r189, Zfp184]","[9930111J21Rik1, 9930111J21Rik2, A530032D15Rik, C130026I21Rik, Fbxo36, Gm7609, Ifi47, Olfr1395, Olfr1396, Olfr56, Slc16a14, Sp100, Sp110, Sp140, Tgtp1, Tgtp2]","[H2-Q1, H2-Q10, H2-Q2, H2-Q4, H2-Q6, H2-Q7, Pou5f1, Tcf19]","[H1f1, H1f2, H1f3, H1f4, H1f6, H2ac4, H2ac6, H2ac7, H2ac8, H2bc3, H2bc4, H2bc6, H2bc7, H2bc8, H3c1, H3c2, H3c3, H3c4, H3c6, H4c1, H4c2, H4c3, H4c4, H4c6, Hfe]",{},"[3632454L22Rik, Armcx1, Armcx2, Armcx3, Armcx4, Armcx6, B230119M05Rik, Hnrnph2, Mir7093, Nxf2, Zmat1]","[A530032D15Rik, C130026I21Rik, Cchcr1, Ccl3, Ccl4, Ccl5, Ccl6, Ccl9, E230016K23Rik, Fbxo36, Gm7609, H2-Q1, H2-Q10, H2-Q2, H2-Q4, H2-Q6, H2-Q7, Pou5f1, Psors1c2, Slc16a14, Sp100, Sp110, Sp140, Tcf19, Wfdc17, Wfdc18]","[9930111J21Rik1, 9930111J21Rik2, A530032D15Rik, C130026I21Rik, Cchcr1, Fbxo36, Gm7609, H2-Q1, H2-Q10, H2-Q2, H2-Q4, H2-Q6, H2-Q7, Ifi47, Olfr1395, Olfr1396, Olfr56, Pou5f1, Psors1c2, Slc16a14, Sp100, Sp110, Sp140, Tcf19, Tgtp1, Tgtp2]","[9930111J21Rik1, 9930111J21Rik2, Ifi47, Irgm1, Olfr1395, Olfr1396, Olfr56, Psme2b, Tgtp1, Tgtp2, Trim7]","[H2ac7, H2ac8, H2bc6, H2bc7, H2bc8, H3c4, H3c6, H4c4]","[Ccl3, Ccl4, Ccl5, Ccl6, Ccl9, E230016K23Rik, H1f1, H1f2, H1f3, H1f4, H1f6, H2ac4, H2ac6, H2ac7, H2ac8, H2bc3, H2bc4, H2bc6, H2bc7, H2bc8, H3c1, H3c2, H3c3, H3c4, H3c6, H3c7, H4c1, H4c2, H4c3, H4c4, H4c6, Hfe, Ifi202b, Ifi203, Ifi204, Ifi207, Ifi208, Ifi209, Ifi211, Ifi213, Mndal, Wfdc17, Wfdc18]"
Gaublomme_GSE75110_TPM_clean,"[9930111J21Rik1, 9930111J21Rik2, H1f1, H1f2, H1f3, H1f4, H1f6, H2ac11, H2ac12, H2ac18, H2ac19, H2ac20, H2ac4, H2ac6, H2ac7, H2ac8, H2bc11, H2bc12, H2bc18, H2bc21, H2bc3, H2bc4, H2bc6, H2bc7, H2bc8, H3c1, H3c13, H3c14, H3c15, H3c2, H3c3, H3c4, H3c6, H3c7, H4c1, H4c14, H4c2, H4c3, H4c4, H4c6, H4c9, Hfe, Ifi47, Irgm1, Olfr1395, Olfr1396, Olfr56, Pom121l2, Prss16, Psme2b, Slc17a2, Tgtp1, Tgtp2, Trim38, Trim41, Trim7, Vmn1r188, Vmn1r189, Zfp184]",{},"[9930111J21Rik1, 9930111J21Rik2, Aoc1, Doxl1, Doxl2, Gimap1, Gimap3, Gimap5, Gimap6, Gimap7, H1f1, H1f2, H1f3, H1f4, H1f6, H2ac11, H2ac12, H2ac18, H2ac19, H2ac20, H2ac4, H2ac6, H2ac7, H2ac8, H2bc11, H2bc12, H2bc18, H2bc21, H2bc3, H2bc4, H2bc6, H2bc7, H2bc8, H3c1, H3c13, H3c14, H3c15, H3c2, H3c3, H3c4, H3c6, H3c7, H4c1, H4c14, H4c2, H4c3, H4c4, H4c6, H4c9, Hfe, Ifi47, Irgm1, Olfr1395, Olfr1396, Olfr56, Pom121l2, Prss16, Psme2b, Slc17a2, Tgtp1, Tgtp2, Tmem176a, Tmem176b, Trim38, Trim41, Trim7, Vmn1r188, Vmn1r189, Zfp184]","[9930111J21Rik1, 9930111J21Rik2, Ifi47, Olfr1395, Olfr1396, Olfr56, Tgtp1, 

In [103]:
chromosome_overlap_upper.applymap(lambda x: len(x) if x is not None else np.nan)

,Gaublomme_GSE75109_TPM_clean,Gaublomme_GSE75110_TPM_clean,Gaublomme_GSE75111_TPM_clean,kakadarov_tpm,somatosensory_converted_into_tpm,Dopaminergic_TPM_clean,Rbp4_positive_cells,Cheng_ES_TPM,Alveolar_cells_Type_II_Merged_Batches,Alveolar_cells_Type_I_Merged_Batches,klein,hepat_TPM_yang_clean,Yu_First_wave_endocrine_cells
Gaublomme_GSE75109_TPM_clean,NaN,34.0,52.0,2.0,1.0,16.0,NaN,4.0,5.0,5.0,3.0,1.0,18.0
Gaublomme_GSE75110_TPM_clean,34.0,NaN,37.0,1.0,1.0,10.0,NaN,NaN,NaN,1.0,5.0,1.0,12.0
Gaublomme_GSE75111_TPM_clean,52.0,37.0,NaN,5.0,1.0,16.0,NaN,NaN,6.0,6.0,3.0,1.0,20.0
kakadarov_tpm,2.0,1.0,5.0,NaN,2.0,12.0,18.0,3.0,23.0,12.0,40.0,5.0,3.0
somatosensory_converted_into_tpm,1.0,1.0,1.0,2.0,NaN,6.0,5.0,NaN,4.0,6.0,6.0,2.0,17.0
Dopaminergic_TPM_clean,16.0,10.0,16.0,12.0,6.0,NaN,NaN,NaN,8.0,4.0,11.0,1.0,11.0
Rbp4_positive_cells,NaN,NaN,NaN,18.0,5.0,NaN,NaN,NaN,5.0,6.0,10.0,NaN,4.0
Cheng_ES_TPM,4.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,6.0,NaN,NaN,3.0
Alveolar_cells_Type_II_Merged_Batches,5.0,NaN,6.0,23.0,4.0,8.0,5.0,3.0,NaN,324.0,71.0,NaN,20.0
Alveolar_cells_Type_I_Merged_Batches,5.0,1.0,6.0,12.0,6.0,4.0,6.0,6.0,324.0,NaN,67.0,NaN,14.0
